# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
USE_GPU = True
device = '/device:GPU:0' if USE_GPU else '/cpu:0'

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы.

In [3]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [4]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y

        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [5]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети.

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [6]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()

    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_TwoLayerFC()

(64, 10)


Реализуйте трехслойную CNN для вашей задачи классификации.

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU
5. Полносвязный слой
6. Функция активации Softmax

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [7]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.conv1 = tf.keras.layers.Conv2D(filters=channel_1, kernel_size=(5,5),
                                            padding='same', activation='relu',
                                            kernel_initializer=initializer)
        self.conv2 = tf.keras.layers.Conv2D(filters=channel_2, kernel_size=(3,3),
                                            padding='same', activation='relu',
                                            kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax',
                                        kernel_initializer=initializer)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################

    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        scores = self.conv1(x)
        scores = self.conv2(scores)
        scores = self.flatten(scores)
        scores = self.fc(scores)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        return scores

In [8]:
def test_ThreeLayerConvNet():
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [11]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.

    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for

    Returns: Nothing, but prints progress during trainingn
    """
    with tf.device(device):


        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

        model = model_init_fn()
        optimizer = optimizer_init_fn()

        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

        t = 0
        for epoch in range(num_epochs):

            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_state()
            train_accuracy.reset_state()

            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:

                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)

                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)

                    if t % print_every == 0:
                        val_loss.reset_state()
                        val_accuracy.reset_state()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)

                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [13]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2
print_every = 100

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.730674982070923, Accuracy: 12.5, Val Loss: 2.834641456604004, Val Accuracy: 13.0
Iteration 100, Epoch 1, Loss: 2.2534189224243164, Accuracy: 27.70730209350586, Val Loss: 1.8840396404266357, Val Accuracy: 37.400001525878906
Iteration 200, Epoch 1, Loss: 2.081246852874756, Accuracy: 31.96517562866211, Val Loss: 1.8338268995285034, Val Accuracy: 40.5
Iteration 300, Epoch 1, Loss: 2.007025718688965, Accuracy: 33.98567199707031, Val Loss: 1.8575338125228882, Val Accuracy: 37.400001525878906
Iteration 400, Epoch 1, Loss: 1.9383149147033691, Accuracy: 35.925811767578125, Val Loss: 1.7416967153549194, Val Accuracy: 42.5
Iteration 500, Epoch 1, Loss: 1.8936060667037964, Accuracy: 36.9323844909668, Val Loss: 1.6745445728302002, Val Accuracy: 41.10000228881836
Iteration 600, Epoch 1, Loss: 1.8632272481918335, Accuracy: 37.79378128051758, Val Loss: 1.6893975734710693, Val Accuracy: 41.5
Iteration 700, Epoch 1, Loss: 1.836732029914856, Accuracy: 38.534236907958984, Val

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 .

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [14]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)


    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.6038882732391357, Accuracy: 7.8125, Val Loss: 8.987314224243164, Val Accuracy: 8.800000190734863
Iteration 100, Epoch 1, Loss: 2.3389625549316406, Accuracy: 22.400989532470703, Val Loss: 1.9214643239974976, Val Accuracy: 32.70000076293945
Iteration 200, Epoch 1, Loss: 2.080897331237793, Accuracy: 28.762439727783203, Val Loss: 1.7189702987670898, Val Accuracy: 39.89999771118164
Iteration 300, Epoch 1, Loss: 1.9515777826309204, Accuracy: 32.194766998291016, Val Loss: 1.6251015663146973, Val Accuracy: 42.0
Iteration 400, Epoch 1, Loss: 1.8531453609466553, Accuracy: 35.13871383666992, Val Loss: 1.5416910648345947, Val Accuracy: 45.400001525878906
Iteration 500, Epoch 1, Loss: 1.7852287292480469, Accuracy: 37.29104232788086, Val Loss: 1.4670937061309814, Val Accuracy: 47.5
Iteration 600, Epoch 1, Loss: 1.7368277311325073, Accuracy: 38.857112884521484, Val Loss: 1.4302096366882324, Val Accuracy: 48.400001525878906
Iteration 700, Epoch 1, Loss: 1.6954145431518555

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [15]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax',
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Iteration 0, Epoch 1, Loss: 3.215442419052124, Accuracy: 4.6875, Val Loss: 2.7577574253082275, Val Accuracy: 13.09999942779541
Iteration 100, Epoch 1, Loss: 2.2206757068634033, Accuracy: 29.331684112548828, Val Loss: 1.9204566478729248, Val Accuracy: 37.900001525878906
Iteration 200, Epoch 1, Loss: 2.064004421234131, Accuracy: 32.96797180175781, Val Loss: 1.8694486618041992, Val Accuracy: 40.20000076293945
Iteration 300, Epoch 1, Loss: 1.9922564029693604, Accuracy: 34.6034049987793, Val Loss: 1.8820695877075195, Val Accuracy: 37.70000076293945
Iteration 400, Epoch 1, Loss: 1.9234411716461182, Accuracy: 36.27649688720703, Val Loss: 1.7320261001586914, Val Accuracy: 42.89999771118164
Iteration 500, Epoch 1, Loss: 1.8797262907028198, Accuracy: 37.337825775146484, Val Loss: 1.6698484420776367, Val Accuracy: 42.69999694824219
Iteration 600, Epoch 1, Loss: 1.851555585861206, Accuracy: 38.18375778198242, Val Loss: 1.717450737953186, Val Accuracy: 41.5
Iteration 700, Epoch 1, Loss: 1.826734542

Альтернативный менее гибкий способ обучения:

In [16]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

766/766 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 2.0172 - sparse_categorical_accuracy: 0.3349 - val_loss: 1.6557 - val_sparse_categorical_accuracy: 0.4530
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.6297 - sparse_categorical_accuracy: 0.4354


[1.638246774673462, 0.43639999628067017]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [19]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    model = tf.keras.Sequential([
      # Первый сверточный слой
      tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
      # Второй сверточный слой
      tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
      tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),

      tf.keras.layers.Flatten(),
      # Полносвязный слой с relu активацией
      tf.keras.layers.Dense(1024, activation='relu'),
      # Выходной слой с softmax активацией
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Iteration 0, Epoch 1, Loss: 2.3339896202087402, Accuracy: 9.375, Val Loss: 2.353342294692993, Val Accuracy: 9.899999618530273
Iteration 100, Epoch 1, Loss: 2.310960531234741, Accuracy: 10.767326354980469, Val Loss: 2.274129629135132, Val Accuracy: 12.399999618530273
Iteration 200, Epoch 1, Loss: 2.2839717864990234, Accuracy: 13.020833015441895, Val Loss: 2.230102777481079, Val Accuracy: 17.899999618530273
Iteration 300, Epoch 1, Loss: 2.262726068496704, Accuracy: 15.214908599853516, Val Loss: 2.197784662246704, Val Accuracy: 21.100000381469727
Iteration 400, Epoch 1, Loss: 2.242192506790161, Accuracy: 17.26932716369629, Val Loss: 2.161949872970581, Val Accuracy: 22.899999618530273
Iteration 500, Epoch 1, Loss: 2.226245641708374, Accuracy: 18.516094207763672, Val Loss: 2.1335840225219727, Val Accuracy: 25.400001525878906
Iteration 600, Epoch 1, Loss: 2.210104465484619, Accuracy: 19.774333953857422, Val Loss: 2.107278347015381, Val Accuracy: 26.0
Iteration 700, Epoch 1, Loss: 2.194254159

In [20]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

766/766 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 1.8532 - sparse_categorical_accuracy: 0.3395 - val_loss: 1.4166 - val_sparse_categorical_accuracy: 0.4890
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.4355 - sparse_categorical_accuracy: 0.4747


[1.441324234008789, 0.4722000062465668]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры.

Ниже представлен пример для полносвязной сети.

In [21]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)

    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)

    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_two_layer_fc_functional()

(64, 10)


In [22]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.1890501976013184, Accuracy: 7.8125, Val Loss: 2.9843106269836426, Val Accuracy: 9.800000190734863
Iteration 100, Epoch 1, Loss: 2.2391374111175537, Accuracy: 28.7592830657959, Val Loss: 1.901845097541809, Val Accuracy: 38.5
Iteration 200, Epoch 1, Loss: 2.070812225341797, Accuracy: 32.77363204956055, Val Loss: 1.8534682989120483, Val Accuracy: 40.400001525878906
Iteration 300, Epoch 1, Loss: 1.9969618320465088, Accuracy: 34.42171859741211, Val Loss: 1.8813104629516602, Val Accuracy: 36.89999771118164
Iteration 400, Epoch 1, Loss: 1.9304864406585693, Accuracy: 36.03881072998047, Val Loss: 1.7551515102386475, Val Accuracy: 39.79999923706055
Iteration 500, Epoch 1, Loss: 1.886325478553772, Accuracy: 37.02594757080078, Val Loss: 1.6786789894104004, Val Accuracy: 42.20000076293945
Iteration 600, Epoch 1, Loss: 1.8565772771835327, Accuracy: 37.88217544555664, Val Loss: 1.6796510219573975, Val Accuracy: 42.599998474121094
Iteration 700, Epoch 1, Loss: 1.830780625

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут).

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [27]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        initializer = tf.initializers.VarianceScaling(scale=2.0)
        # Блок 1
        self.conv1 = tf.keras.layers.Conv2D(64, (3,3), padding='same', kernel_initializer=initializer)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.relu1 = tf.keras.layers.ReLU()
        # Блок 2
        self.conv2 = tf.keras.layers.Conv2D(64, (3,3), padding='same', kernel_initializer=initializer)
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.relu2 = tf.keras.layers.ReLU()

        self.pool1 = tf.keras.layers.MaxPool2D((2,2))
        self.dropout1 = tf.keras.layers.Dropout(0.25)
        # Блок 3
        self.conv3 = tf.keras.layers.Conv2D(128, (3,3), padding='same', kernel_initializer=initializer)
        self.bn3 = tf.keras.layers.BatchNormalization()
        self.relu3 = tf.keras.layers.ReLU()
        # Блок 4
        self.conv4 = tf.keras.layers.Conv2D(128, (3,3), padding='same', kernel_initializer=initializer)
        self.bn4 = tf.keras.layers.BatchNormalization()
        self.relu4 = tf.keras.layers.ReLU()

        self.pool2 = tf.keras.layers.MaxPool2D((2,2))
        self.dropout2 = tf.keras.layers.Dropout(0.35)
        # Блок 5
        self.conv5 = tf.keras.layers.Conv2D(256, (3,3), padding='same', kernel_initializer=initializer)
        self.bn5 = tf.keras.layers.BatchNormalization()
        self.relu5 = tf.keras.layers.ReLU()

        self.pool3 = tf.keras.layers.MaxPool2D((2,2))
        self.dropout3 = tf.keras.layers.Dropout(0.4)

        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(512, activation='relu', kernel_initializer=initializer)
        self.bnfc = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.ReLU()
        self.dropout_fc = tf.keras.layers.Dropout(0.5)
        self.fc_out = tf.keras.layers.Dense(10, activation='softmax', kernel_initializer=initializer)


        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################

    def call(self, x, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x =  self.relu2(x)
        x = self.pool1(x)
        if training:
          x = self.dropout1(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.pool2(x)
        if training:
          x = self.dropout2(x)

        x = self.conv5(x)
        x = self.bn5(x)
        x = self.relu5(x)
        x = self.pool3(x)
        if training:
          x = self.dropout3(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = self.bnfc(x)
        x = self.relu(x)
        if training:
          x = self.dropout_fc(x)
        x = self.fc_out(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 3.6561079025268555, Accuracy: 9.375, Val Loss: 2.998950242996216, Val Accuracy: 12.199999809265137
Iteration 700, Epoch 1, Loss: 1.5161646604537964, Accuracy: 46.62535858154297, Val Loss: 1.1695425510406494, Val Accuracy: 61.79999542236328
Iteration 1400, Epoch 2, Loss: 1.0199286937713623, Accuracy: 63.94684982299805, Val Loss: 0.8856332898139954, Val Accuracy: 69.5
Iteration 2100, Epoch 3, Loss: 0.8502669334411621, Accuracy: 69.88961029052734, Val Loss: 0.7688243389129639, Val Accuracy: 73.4000015258789
Iteration 2800, Epoch 4, Loss: 0.7535499930381775, Accuracy: 73.43439483642578, Val Loss: 0.7057284116744995, Val Accuracy: 76.20000457763672
Iteration 3500, Epoch 5, Loss: 0.688689649105072, Accuracy: 75.71867370605469, Val Loss: 0.6174358129501343, Val Accuracy: 78.30000305175781
Iteration 4200, Epoch 6, Loss: 0.6398293972015381, Accuracy: 77.73753356933594, Val Loss: 0.6216412782669067, Val Accuracy: 78.19999694824219
Iteration 4900, Epoch 7, Loss: 0.5924

Удалось достичь точности на вадидационной выборке в 83.7%

Реализована улучшенная сверточная сеть, включающая:
- 5 свёрточных слоёв
- BatchNormalization после каждого сверточного слоя
- MaxPooling слои
- Dropout для регуляризации
- Полносвязный блок: Dense(512) + - Dropout
- Выходной слой Softmax
Архитектура похожа на VGG
CustomConvNet показала наилучшую точность благодаря глубокой архитектуре, BatchNorm и Dropout.

При одинаковых гирерпараметрах были протестированы оптимизаторы:  Adam, SGD, RMSprop. SGD показал наихудшие результаты - за 10 эпох точность на валидационной выборке была в районе 69%. Adam и RMSprop показали примерно одинаковые результаты, однако Adam был немножко точнее.

Наилучший результат был получен пи learning rate = 1e-3, при 1e-4 наблюдалась более медленная сходимость и меньшая точность модели после 10 эпох обучения (78,3%)